## 흉부 x-ray 사진을 인경신공망 학습을 이용하여 폐렴 환자 구분 하기

### 기본 라이브러리 임포트

In [1]:
########################################
##### 기본적으로 필요한 라이브러리 import #####
########################################
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import cv2
import skimage
import numpy as np
from glob import glob
import matplotlib.cm as cm

########################################
##### 머신러닝 관련 라이브러리 import #####
########################################
import sklearn
import scipy
from skimage.transform import resize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 데이터 경로 설정

In [2]:
train_dir = "../chest_xray/train/"
test_dir =  "../chest_xray/test/" 

### 데이터 크기 설정

In [3]:
# 데이터 크기 (150 x 150)
_N_ROW = 150  # 세로 150
_N_COL = 150  # 가로 150
_N_PIXEL = _N_ROW * _N_COL

In [4]:
# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(_N_ROW, _N_COL),
        batch_size=100,
        class_mode='categorical')

Found 5216 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(_N_ROW, _N_COL),
        batch_size=100,
        class_mode='categorical') 

Found 624 images belonging to 2 classes.


### 에러 확인 함수 생성

In [6]:
def confirmErr(realAnswerList, testAnswerList):
    answerNum = 0
    for i, eachAns in enumerate(realAnswerList):
        if testAnswerList[i] == eachAns:
            answerNum += 1
    return float(len(realAnswerList)-answerNum)/float(len(realAnswerList))*100

### 모델 구상

In [7]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(_N_ROW,_N_COL,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

### 모델 학습과정 설정

In [8]:
# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. 모델 학습시키기

In [9]:
# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=5,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/5
50/50 [==============================] - 488s 10s/step - loss: 4.0550 - acc: 0.7440 - val_loss: 5.9315 - val_acc: 0.6320
Epoch 2/5
50/50 [==============================] - 485s 10s/step - loss: 4.0774 - acc: 0.7470 - val_loss: 5.9315 - val_acc: 0.6320
Epoch 3/5
50/50 [==============================] - 484s 10s/step - loss: 4.2175 - acc: 0.7383 - val_loss: 5.9315 - val_acc: 0.6320
Epoch 4/5
50/50 [==============================] - 544s 11s/step - loss: 4.1152 - acc: 0.7447 - val_loss: 5.9315 - val_acc: 0.6320
Epoch 5/5
50/50 [==============================] - 642s 13s/step - loss: 4.0447 - acc: 0.7491 - val_loss: 5.9315 - val_acc: 0.6320


### 6. 모델 평가하기

In [13]:
# 6. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=15)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 62.76%


### 7. 모델 사용하기

In [12]:
# 7. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=15)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'NORMAL': 0, 'PNEUMONIA': 1}
[[0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 ...
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]]
